In [1]:
import numpy as np
import pandas as pd
import scipy 
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [2]:
pwd

'/Users/ijyapaudel/Desktop'

In [3]:
cd Thinkful

/Users/ijyapaudel/Desktop/Thinkful


In [4]:
cd Thinkful

[Errno 2] No such file or directory: 'Thinkful'
/Users/ijyapaudel/Desktop/Thinkful


In [5]:
cd Bayes

/Users/ijyapaudel/Desktop/Thinkful/Bayes


In [6]:
df = pd.read_csv("amazon.txt",delimiter="\t", header = None)
df.columns = ['comment', 'rating']

In [7]:
df.head()

,comment,rating
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [8]:
print("Schema:\n\n",df.dtypes)
print("Number of questions,columns=",df.shape)

Schema:

 comment    object
rating      int64
dtype: object
Number of questions,columns= (1000, 2)


In [9]:
df['comment'] = df.comment.map(lambda x: x.lower())

In [10]:
df['comment'] = df.comment.str.replace('[^\w\s]', '') 

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
df['comment'] = df['comment'].apply(lambda x: ''.join(x))
count_vect = CountVectorizer()
counts = count_vect.fit_transform(df['comment'])

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer().fit(counts)

counts = transformer.transform(counts)  

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, df['rating'], test_size=0.1, random_state=69)  

In [14]:
from sklearn.naive_bayes import BernoulliNB

In [15]:
model = BernoulliNB().fit(X_train, y_train)

In [16]:
predicted = model.predict(X_test)
print(np.mean(predicted == y_test))

0.79


In [17]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, predicted))  


[[32 12]
 [ 9 47]]


In [22]:
keywords = ['problem', 'waste', 'will not', 'did not', 'have not', 'unfortunately', 'disappointing', 'awful', 'worst', 'fooled', 'odd', 'crap', 'waste', 'sad', 'poor', 'angry', 'lack', 'wasted', 'loss', 'bad', 'horrible', 'return', 'fool', 'misleading', 'junk', 'disappointed', 'disappointed', 'painful', 'missing', 'cannot', 'didn\'t', 'negative', 'bad', 
        'unsatisfactory', 'only', 'does not match', 'can not', 'flops']

In [23]:
for key in keywords:
    df[str(key)]= df.comment.str.contains(' ' + str(key)+' ', case = False)

In [24]:
data = df[keywords]
target = df['rating']

In [25]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
bnb.fit(data, target)
y_pred = bnb.predict(data)
print("Number of mislabeld points of total {} points: {}".format(data.shape[0], (target != y_pred).sum()))


Number of mislabeld points of total 1000 points: 423


In [28]:
#Hold Out Example
from sklearn.model_selection import train_test_split
# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(bnb.fit(X_train, y_train).score(X_test, y_test)))
print('Testing on Sample: ' + str(bnb.fit(data, target).score(data, target)))

With 20% Holdout: 0.59
Testing on Sample: 0.577


In [29]:
#Not too much overfitting cause our data is fairly simple in nature
from sklearn.model_selection import cross_val_score
cross_val_score(bnb, data, target, cv=10)

array([0.56, 0.52, 0.61, 0.6 , 0.58, 0.56, 0.55, 0.54, 0.56, 0.61])